# <center>CSCE4143-Final-Project</center>
##### <center>San Francisco Crime Classification</center>
##### <center>Group4: David Hammons | Logan Milazzo | Jose Martinez | Steve Liang</center>

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn import preprocessing
from zipfile import ZipFile

1. SVM
2. Linear Regression
3. Decision Tree

### Data Cleaning

In [4]:
! chmod 600 kaggle.json && (ls ~/.kaggle 2>/dev/null || mkdir ~/.kaggle) && cp kaggle.json ~/.kaggle/ && echo 'Done'
! kaggle competitions download -c sf-crime
! ls

kaggle.json
Done
sf-crime.zip: Skipping, found more recently modified local copy (use --force to force download)
CSCE4143-Final-Project.ipynb kaggle.json
README.md                    sf-crime.zip


In [5]:
data_zip = ZipFile("sf-crime.zip")
train_df = pd.read_csv(data_zip.open("sf-crime/train.csv"))
test_df = pd.read_csv(data_zip.open("sf-crime/test.csv"))

In [6]:
train_df.tail(10)

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
878039,2003-01-06 00:40:00,OTHER OFFENSES,"DRIVERS LICENSE, SUSPENDED OR REVOKED",Monday,NORTHERN,"ARREST, CITED",POLK ST / CALIFORNIA ST,-122.420692,37.790577
878040,2003-01-06 00:33:00,ASSAULT,INFLICT INJURY ON COHABITEE,Monday,MISSION,NONE,2800 Block of FOLSOM ST,-122.414073,37.751685
878041,2003-01-06 00:31:00,OTHER OFFENSES,"DRIVERS LICENSE, SUSPENDED OR REVOKED",Monday,RICHMOND,"ARREST, CITED",CLEMENT ST / 14TH AV,-122.472985,37.782552
878042,2003-01-06 00:20:00,ASSAULT,ATTEMPTED HOMICIDE WITH A GUN,Monday,BAYVIEW,"ARREST, BOOKED",1500 Block of SHAFTER AV,-122.389769,37.730564
878043,2003-01-06 00:20:00,OTHER OFFENSES,PAROLE VIOLATION,Monday,BAYVIEW,"ARREST, BOOKED",1500 Block of SHAFTER AV,-122.389769,37.730564
878044,2003-01-06 00:15:00,ROBBERY,ROBBERY ON THE STREET WITH A GUN,Monday,TARAVAL,NONE,FARALLONES ST / CAPITOL AV,-122.459033,37.714056
878045,2003-01-06 00:01:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Monday,INGLESIDE,NONE,600 Block of EDNA ST,-122.447364,37.731948
878046,2003-01-06 00:01:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Monday,SOUTHERN,NONE,5TH ST / FOLSOM ST,-122.403390,37.780266
878047,2003-01-06 00:01:00,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM OF VEHICLES",Monday,SOUTHERN,NONE,TOWNSEND ST / 2ND ST,-122.390531,37.780607
878048,2003-01-06 00:01:00,FORGERY/COUNTERFEITING,"CHECKS, FORGERY (FELONY)",Monday,BAYVIEW,NONE,1800 Block of NEWCOMB AV,-122.394926,37.738212


In [7]:
drop_col = ["Resolution", "Descript"]
train_df.drop(drop_col, axis=1, inplace=True)
train_df

,Dates,Category,DayOfWeek,PdDistrict,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,Wednesday,NORTHERN,VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,Wednesday,NORTHERN,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,Wednesday,PARK,100 Block of BRODERICK ST,-122.438738,37.771541
...,...,...,...,...,...,...,...
878044,2003-01-06 00:15:00,ROBBERY,Monday,TARAVAL,FARALLONES ST / CAPITOL AV,-122.459033,37.714056
878045,2003-01-06 00:01:00,LARCENY/THEFT,Monday,INGLESIDE,600 Block of EDNA ST,-122.447364,37.731948
878046,2003-01-06 00:01:00,LARCENY/THEFT,Monday,SOUTHERN,5TH ST / FOLSOM ST,-122.403390,37.780266
878047,2003-01-06 00:01:00,VANDALISM,Monday,SOUTHERN,TOWNSEND ST / 2ND ST,-122.390531,37.780607


### check NULL cells

In [8]:
print(f"train null: {train_df.isna().sum()}")
print(f"test null: {test_df.isna().sum()}")

train null: Dates         0
Category      0
DayOfWeek     0
PdDistrict    0
Address       0
X             0
Y             0
dtype: int64
test null: Id            0
Dates         0
DayOfWeek     0
PdDistrict    0
Address       0
X             0
Y             0
dtype: int64


### One Hot Encoding

In [9]:
le = preprocessing.LabelEncoder()
train_df = train_df.apply(le.fit_transform)
test_df = test_df.apply(le.fit_transform)
train_df

,Dates,Category,DayOfWeek,PdDistrict,Address,X,Y
0,389256,37,6,4,19790,18787,23130
1,389256,21,6,4,19790,18787,23130
2,389255,21,6,4,22697,19317,32784
3,389254,16,6,4,4266,18403,32936
4,389254,16,6,5,1843,13848,21892
...,...,...,...,...,...,...,...
878044,1,25,1,8,15816,8189,1487
878045,0,16,1,2,11491,10741,7323
878046,0,16,1,7,11315,29074,25702
878047,0,35,1,7,22308,32850,25848


In [10]:
target_col = "Category"
feature_col = ["Dates", "Category", "DayOfWeek", "PdDistrict", "Address", "X", "Y"]
target_col, feature_col

('Category',
 ['Dates', 'Category', 'DayOfWeek', 'PdDistrict', 'Address', 'X', 'Y'])

In [11]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

train_df[feature_col] = scaler.fit_transform(train_df[feature_col])

x_train, y_train = train_df[feature_col], train_df[target_col]

x_train, y_train

(           Dates  Category  DayOfWeek  PdDistrict   Address         X  \
 0       1.712912  1.652346   1.485499   -0.155501  1.702741 -0.285690   
 1       1.712912  0.155428   1.485499   -0.155501  1.702741 -0.285690   
 2       1.712903  0.155428   1.485499   -0.155501  2.182786 -0.222699   
 3       1.712894 -0.312359   1.485499   -0.155501 -0.860805 -0.331329   
 4       1.712894 -0.312359   1.485499    0.193696 -1.260925 -0.872695   
 ...          ...       ...        ...         ...       ...       ...   
 878044 -1.713552  0.529657  -0.982295    1.241287  1.046497 -1.545273   
 878045 -1.713560 -0.312359  -0.982295   -0.853895  0.332291 -1.241965   
 878046 -1.713560 -0.312359  -0.982295    0.892090  0.303227  0.936931   
 878047 -1.713560  1.465231  -0.982295    0.892090  2.118549  1.385712   
 878048 -1.713560 -0.686588  -0.982295   -1.552289 -0.718459  1.272209   
 
                Y  
 0       0.275005  
 1       0.275005  
 2       1.346753  
 3       1.363628  
 4       0

### Model

In [12]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation=tf.nn.relu, input_shape=(x_train.columns.size, )),
    tf.keras.layers.Dense(32, activation=tf.nn.relu),
    tf.keras.layers.Dense(16, activation=tf.nn.relu),
    tf.keras.layers.Dense(64, activation=tf.nn.sigmoid)
])

In [13]:
model.compile(
    loss = 'binary_crossentropy',
    optimizer = "Adam",
    metrics = ['accuracy']
)

In [14]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor = 'loss',
    min_delta = 1e-3,
    patience = 5
)

In [15]:
final_model = model.fit(
    x_train,
    y_train,
    epochs = 150,
    verbose = 2,
    callbacks = [early_stopping]
)

final_model.history['accuracy'][-1]

: 

: 